在了解完CNN的基本概念之后，本节我们会带领大家用keras来实现一个简单的卷积神经网络，使用的数据是MNIST，MNIST 数据集来自美国国家标准与技术研究所, 数据集由来自 250 个不同人手写的数字构成,包括0-9共10个数字，我们的任务是正确识别出手写数字。本节将会构建一个非常简单并且有代表性的卷及神经网络，预期可达到99%的准确率，读者可通过该例子掌握keras搭建卷及神经网络的要点。

首先载入MNIST数据集，这里直接采用tensorflow内置的获取mnist数据的方法，该方法会把数据下载到对应的目录中，因此执行以下方法时读者需等待片刻。当然读者也可自行下载，地址为：http://yann.lecun.com/exdb/mnist/ 注意，这里在读取数据时read_data_sets方法的one_hot参数必须设置为True,因为我们这里是一个10分类的问题，label标签要求是独热编码，如果用户自行下载数据注意把label标签做one-hot处理。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist/', one_hot=True)
train_X = mnist.train.images
train_y = mnist.train.labels
test_X = mnist.test.images
test_y = mnist.test.labels

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


数据读取完成后，我们对数据的维度进行查看。

In [4]:
train_X.shape

(55000, 784)

可以看到，我们读取的数据是二维的，第一个维度表示的是图片的数量，第二个维度是图片的像素集，我们需要把这一维的数据转换为一个28×28×1的数据集，其中28表示图片的长宽，由于图片是单色道即黑白图片，因此第三个维度是1。我们这里使用reshape方法来改变数据维度。

In [5]:
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

接下来就是建模的过程，由于模型比较简单，我们这里采用序贯模型Sequential。首先，我们创建第一个卷基层，其中filters表示卷积核的个数，kernel_size表示卷积核的大小，padding表示填充方式，其中包括“valid”与“same”，“valid”代表只进行有效的卷积，即对边界数据不处理，“same”代表保留边界处的卷积结果，通常会导致输出shape与输入shape相同，input_shape表示输入数据的维度，注意，如果是第一层卷基层，必须提供该参数。接下来我们引入一些非线性的变化操作添加一个relu激活函数。最后，使用一个2×2的最大池化层对卷积的输出结果做池化操作，其中pool_size表示赤化层的大小。

In [6]:
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Dense, Conv2D
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5),padding='valid',input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

Using TensorFlow backend.


然后我们定义我们的第二个池化层，这个卷基层和第一层类似，只是卷积核的数量我们改为了64。

In [7]:
model.add(Conv2D(filters=64, kernel_size=(5, 5),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

在经过上面两个卷基层之后，我们的图片大小最后为5×5，由于最后一个卷基层的卷积核是64，所以我们最后拿到的输出结果维度是5×5×64。为了方便加入后面的分类，我们这里加入一个Flatten层，Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。然后连接一个全连接层，并加入relu激活函数，为了防止过拟合，我们添加一个Dropout层，然后我们连接一个10维的全连接层，最后我们把得到的结果输入到softmax层，从而得到最后的概率。

In [8]:
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

模型建立完成之后，我们来定义损失函数，优化算法与评估指标，其中损失函数我们采用多分类常使用的croos entropy，优化方法我们采用adam，评估指标是准确率。

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

下面就是我们的训练过程了，其中batch_size表示mini-batch的大小，epochs表示训练的迭代次数，verbose表示是否打印日志，0为不在标准输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行记录。

In [11]:
model.fit(train_X, train_y, batch_size=128, epochs=10, verbose=1)

Epoch 1/10
55000/55000 [==============================] - 3s - loss: 0.3012 - acc: 0.9071     
Epoch 2/10
55000/55000 [==============================] - 1s - loss: 0.0878 - acc: 0.9744     
Epoch 3/10
55000/55000 [==============================] - 1s - loss: 0.0616 - acc: 0.9821     
Epoch 4/10
55000/55000 [==============================] - 2s - loss: 0.0477 - acc: 0.9857     
Epoch 5/10
55000/55000 [==============================] - 1s - loss: 0.0402 - acc: 0.9881     
Epoch 6/10
55000/55000 [==============================] - 2s - loss: 0.0324 - acc: 0.9901     
Epoch 7/10
55000/55000 [==============================] - 2s - loss: 0.0304 - acc: 0.9908     
Epoch 8/10
55000/55000 [==============================] - 2s - loss: 0.0261 - acc: 0.9919     
Epoch 9/10
55000/55000 [==============================] - 2s - loss: 0.0232 - acc: 0.9925     
Epoch 10/10
55000/55000 [==============================] - 1s - loss: 0.0213 - acc: 0.9937     


训练完成后，我们可以在测试集上进行测试，得到最终的测试结果。

In [12]:
score = model.evaluate(test_X, test_y, batch_size=128, verbose=0)

print('Test accuracy:', score[1])

Test accuracy: 0.9923


可以看到，我们这么简单的一个卷积神经网络，准确率已经高达99.37%，可见卷积层对图像的特征提取是十分有用的，并且依靠卷积核的参数共享，训练效率得到了很大的提升。在后面的文章中，我们会给大家介绍一些更加复杂且效果更好的卷积神经网络，这些网络在各大比赛中都取得了优异的成绩，读者可借鉴这些网络的优点来搭建自己的神经网络，当然也可以采用迁移学习的方法直接使用预先训练好的模型来完成自己的图像识别任务。